In [1]:
import mysql.connector
import pandas as pd
import numpy as np
# Database configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'durus1234',
    'database': 'tfm'
}

def get_motor_entries_df(motor_id):
    # Create a connection to the database
    try:
        connection = mysql.connector.connect(**db_config)
        query = """
        SELECT * FROM motor
        WHERE id = %s
        """
        
        # Load the query result into a DataFrame
        df = pd.read_sql(query, connection, params=(motor_id,))

        # Close the connection
        connection.close()

        return df

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

# Example usage
motor_id = 3  # Replace with the actual motor_id you want to query
df = get_motor_entries_df(motor_id)

df.head()

C:\Users\daric\AppData\Local\Temp\ipykernel_16840\597328281.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection, params=(motor_id,))


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,3,1,-0.0001,0.0001,100.0,518.67,642.03,1589.92,1408.39,14.62,...,522.20,2388.13,8131.17,8.4330,0.03,392,2388,100.0,38.99,23.2960
1,3,2,0.0039,-0.0003,100.0,518.67,642.23,1597.31,1407.90,14.62,...,521.19,2388.08,8132.69,8.4183,0.03,393,2388,100.0,38.84,23.3191
2,3,3,0.0006,0.0003,100.0,518.67,642.98,1586.77,1407.58,14.62,...,521.92,2388.09,8131.29,8.4017,0.03,391,2388,100.0,38.69,23.3774
3,3,4,-0.0011,-0.0001,100.0,518.67,642.42,1590.52,1401.42,14.62,...,521.59,2388.09,8134.41,8.4398,0.03,395,2388,100.0,38.72,23.3676
4,3,5,-0.0004,-0.0003,100.0,518.67,642.48,1583.09,1405.87,14.62,...,521.37,2388.09,8135.05,8.4540,0.03,392,2388,100.0,38.81,23.2853


In [2]:
df.shape

(126, 26)

In [3]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Suponiendo que df es tu DataFrame
df['cycle_norm'] = df['cycle']
cols_to_normalize = df.columns.difference(['id', 'cycle'])

# Definir el pipeline para normalización
pipeline = Pipeline(steps=[
    ('scaler', MinMaxScaler())  # Escalar las columnas especificadas
])

# Aplicar el pipeline para normalizar las columnas especificadas
transformed_array = pipeline.fit_transform(df[cols_to_normalize])

# Convertir el array transformado de nuevo a un DataFrame
transformed_df = pd.DataFrame(transformed_array, columns=cols_to_normalize)

# Añadir las columnas no normalizadas de nuevo al DataFrame
transformed_df['id'] = df['id']
transformed_df['cycle'] = df['cycle']


# Reordenar columnas para que tengan el mismo orden que en el DataFrame original
# Es importante que las columnas de `cols_to_normalize` sean las mismas en el orden original
final_columns = ['id', 'cycle'] + list(cols_to_normalize)
transformed_df = transformed_df[final_columns]

# Mostrar las primeras filas del DataFrame transformado
transformed_df.head()

,id,cycle,cycle_norm,s1,s10,s11,s12,s13,s14,s15,...,s3,s4,s5,s6,s7,s8,s9,setting1,setting2,setting3
0,3,1,0.000,0.0,0.0,0.706897,1.000000,0.529412,0.586184,0.366222,...,0.464662,0.513708,0.0,0.0,0.556522,0.1250,0.652028,0.483221,0.6,0.0
1,3,2,0.008,0.0,0.0,0.241379,0.512077,0.235294,0.686184,0.235556,...,0.854430,0.491685,0.0,0.0,0.704348,0.3750,0.143455,0.751678,0.2,0.0
2,3,3,0.016,0.0,0.0,0.172414,0.864734,0.294118,0.594079,0.088000,...,0.298523,0.477303,0.0,0.0,0.482609,0.0625,0.641154,0.530201,0.8,0.0
3,3,4,0.024,0.0,0.0,0.620690,0.705314,0.294118,0.799342,0.426667,...,0.496308,0.200449,0.0,0.0,0.517391,0.1875,0.542451,0.416107,0.4,0.0
4,3,5,0.032,0.0,0.0,0.413793,0.599034,0.294118,0.841447,0.552889,...,0.104430,0.400449,0.0,0.0,0.647826,0.6875,0.695943,0.463087,0.2,0.0


In [5]:
import tensorflow as tf
import joblib


# Paths to the model files
model1_path = 'C:/Users/daric/OneDrive/Desktop/tfm_v2/models/model_rf_binary_smote.joblib'  # Corrected file name for scikit-learn model
model3_path = 'C:/Users/daric/OneDrive/Desktop/tfm_v2/models/model3_lstm_RUL.keras'
rf_model_path = 'C:/Users/daric/OneDrive/Desktop/tfm_v2/models/model_rf_multiclassifier_smote.joblib'

# Load Keras models
try:
    model3_lstm_RUL = tf.keras.models.load_model(model3_path)
    print(f"model3_lstm_RUL loaded successfully from {model3_path}.")
except Exception as e:
    print(f"Error loading model3_lstm_RUL: {e}")

# Load scikit-learn models
try:
    model_rf_binary_smote = joblib.load(model1_path)
    print(f"model_rf_binary_smote loaded successfully from {model1_path}.")
except Exception as e:
    print(f"Error loading model_rf_binary_smote: {e}")

try:
    model_rf_multiclassifier_smote = joblib.load(rf_model_path)
    print(f"model_rf_multiclassifier_smote loaded successfully from {rf_model_path}.")
except Exception as e:
    print(f"Error loading model_rf_multiclassifier_smote: {e}")


model3_lstm_RUL loaded successfully from C:/Users/daric/OneDrive/Desktop/tfm_v2/models/model3_lstm_RUL.keras.
model_rf_binary_smote loaded successfully from C:/Users/daric/OneDrive/Desktop/tfm_v2/models/model_rf_binary_smote.joblib.
model_rf_multiclassifier_smote loaded successfully from C:/Users/daric/OneDrive/Desktop/tfm_v2/models/model_rf_multiclassifier_smote.joblib.


In [6]:
transformed_df.shape

(126, 27)

In [7]:
df.shape

(126, 27)

In [23]:
sequence_length = 50
sequence_cols = [
    'setting1', 'setting2', 'setting3', 'cycle_norm',
    's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9',
    's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17',
    's18', 's19', 's20', 's21'
]

def prepare_sequences_for_prediction(df, sequence_length, sequence_cols):
    sequences = []
    # Create sequences for LSTM prediction
    for i in range(len(df) - sequence_length + 1):
        seq = df[sequence_cols].iloc[i:i + sequence_length].values
        sequences.append(seq)
    return np.array(sequences)

def make_lstm_predictions(model, df, sequence_length, sequence_cols):
    # Check if there are enough rows to create sequences
    if len(df) < sequence_length:
        raise ValueError(f"Not enough data to create sequences. Need at least {sequence_length} rows.")

    # Prepare sequences from the DataFrame
    X_sequences = prepare_sequences_for_prediction(df, sequence_length, sequence_cols)
    
    # Predict with the LSTM model
    lstm_predictions = model.predict(X_sequences)
    
    # Ensure predictions are in the expected shape
    if lstm_predictions.ndim == 3:
        lstm_predictions = np.squeeze(lstm_predictions, axis=1)
    
    # Initialize prediction column
    df['RUL_PREDICTION'] = np.nan
    
    # Fill predictions into the DataFrame
    for i in range(len(lstm_predictions)):
        df.loc[i + sequence_length - 1, 'RUL_PREDICTION'] = lstm_predictions[i]
    
    return df

# Apply predictions to transformed_df
try:
    transformed_df_with_predictions = make_lstm_predictions(model3_lstm_RUL, transformed_df, sequence_length, sequence_cols)
    print(transformed_df_with_predictions.head())
except ValueError as e:
    print(f"Error: {e}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
   id  cycle  cycle_norm   s1  s10       s11       s12       s13       s14  \
0   3      1       0.000  0.0  0.0  0.706897  1.000000  0.529412  0.586184   
1   3      2       0.008  0.0  0.0  0.241379  0.512077  0.235294  0.686184   
2   3      3       0.016  0.0  0.0  0.172414  0.864734  0.294118  0.594079   
3   3      4       0.024  0.0  0.0  0.620690  0.705314  0.294118  0.799342   
4   3      5       0.032  0.0  0.0  0.413793  0.599034  0.294118  0.841447   

        s15  ...        s7      s8        s9  setting1  setting2  setting3  \
0  0.366222  ...  0.556522  0.1250  0.652028  0.483221       0.6       0.0   
1  0.235556  ...  0.704348  0.3750  0.143455  0.751678       0.2       0.0   
2  0.088000  ...  0.482609  0.0625  0.641154  0.530201       0.8       0.0   
3  0.426667  ...  0.517391  0.1875  0.542451  0.416107       0.4       0.0   
4  0.552889  ...  0.647826  0.6875  0.695943  0.463087       0.2       0.0   

   lstm_rul_predictions 

In [22]:
transformed_df_with_predictions.head()

,id,cycle,cycle_norm,s1,s10,s11,s12,s13,s14,s15,...,s7,s8,s9,setting1,setting2,setting3,lstm_rul_predictions,rf_binary_predictions,rf_multiclass_predictions,RUL_PREDICTION
0,3,1,0.000,0.0,0.0,0.706897,1.000000,0.529412,0.586184,0.366222,...,0.556522,0.1250,0.652028,0.483221,0.6,0.0,4.48739,0,0,NaN
1,3,2,0.008,0.0,0.0,0.241379,0.512077,0.235294,0.686184,0.235556,...,0.704348,0.3750,0.143455,0.751678,0.2,0.0,4.48739,0,0,NaN
2,3,3,0.016,0.0,0.0,0.172414,0.864734,0.294118,0.594079,0.088000,...,0.482609,0.0625,0.641154,0.530201,0.8,0.0,4.48739,0,0,NaN
3,3,4,0.024,0.0,0.0,0.620690,0.705314,0.294118,0.799342,0.426667,...,0.517391,0.1875,0.542451,0.416107,0.4,0.0,4.48739,0,0,NaN
4,3,5,0.032,0.0,0.0,0.413793,0.599034,0.294118,0.841447,0.552889,...,0.647826,0.6875,0.695943,0.463087,0.2,0.0,4.48739,0,0,NaN


In [24]:

def prepare_lstm_input(df, sequence_length=50):
    required_features = [
        'setting1', 'setting2', 'setting3', 'cycle_norm',
        's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9',
        's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17',
        's18', 's19', 's20', 's21'
    ]
    
    missing_cols = [col for col in required_features if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing columns in input DataFrame: {missing_cols}")
    
    df = df[required_features]
    
    if df.shape[0] < sequence_length:
        padding = np.zeros((sequence_length - df.shape[0], len(required_features)))
        df_padded = np.vstack([padding, df.values])
    else:
        df_padded = df.values[-sequence_length:]
    
    X_lstm = np.expand_dims(df_padded, axis=0)
    
    return X_lstm

def make_predictions(transformed_df, sequence_length=50):
    required_features = [
        'setting1', 'setting2', 'setting3', 'cycle_norm',
        's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9',
        's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17',
        's18', 's19', 's20', 's21'
    ]
    
    if transformed_df.shape[0] < sequence_length:
        raise ValueError(f"DataFrame has only {transformed_df.shape[0]} rows, but at least {sequence_length} are required.")
    
    # Prepare input for LSTM models
    X_lstm = prepare_lstm_input(transformed_df, sequence_length)
    
    # Predict with LSTM models
    lstm_rul_predictions = model3_lstm_RUL.predict(X_lstm)
    
    # Handle single sample batch case
    if lstm_rul_predictions.ndim == 3:
        lstm_rul_predictions = np.squeeze(lstm_rul_predictions, axis=0)
    
    # Repeat the LSTM predictions if necessary
    if lstm_rul_predictions.ndim == 1:
        lstm_rul_predictions = np.repeat(lstm_rul_predictions, len(transformed_df))
    elif len(lstm_rul_predictions) != len(transformed_df):
        raise ValueError(f"Length of LSTM RUL predictions ({len(lstm_rul_predictions)}) does not match length of DataFrame ({len(transformed_df)})")
    
    # Prepare input for Random Forest models
    rf_feature_cols = required_features
    if all(col in transformed_df.columns for col in rf_feature_cols):
        X_rf = transformed_df[rf_feature_cols].copy()  # Ensure it's a DataFrame
    else:
        raise ValueError("Input DataFrame does not contain all required feature columns for Random Forest models.")
    
    # Predict with Random Forest models
    rf_binary_predictions = model_rf_binary_smote.predict(X_rf)
    rf_multiclass_predictions = model_rf_multiclassifier_smote.predict(X_rf)
    
    # Ensure predictions are the correct length
    if len(rf_binary_predictions) != len(transformed_df):
        raise ValueError(f"Length of Random Forest binary predictions ({len(rf_binary_predictions)}) does not match length of DataFrame ({len(transformed_df)})")
    if len(rf_multiclass_predictions) != len(transformed_df):
        raise ValueError(f"Length of Random Forest multiclass predictions ({len(rf_multiclass_predictions)}) does not match length of DataFrame ({len(transformed_df)})")

    # Add predictions to the DataFrame
    transformed_df['RUL_PREDICTION'] = lstm_rul_predictions
    transformed_df['BINARY_CLASSIFICATION'] = rf_binary_predictions
    transformed_df['MULTICLASS_CLASSIFICATION'] = rf_multiclass_predictions

    return transformed_df

# Example usage
try:
    transformed_df_with_predictions = make_predictions(transformed_df, sequence_length=50)
    print(transformed_df_with_predictions.head())
except ValueError as e:
    print(f"Error: {e}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Error: Length of LSTM RUL predictions (1) does not match length of DataFrame (126)


In [25]:
transformed_df_with_predictions.head(30)

,id,cycle,cycle_norm,s1,s10,s11,s12,s13,s14,s15,...,s7,s8,s9,setting1,setting2,setting3,lstm_rul_predictions,rf_binary_predictions,rf_multiclass_predictions,RUL_PREDICTION
0,3,1,0.000,0.0,0.0,0.706897,1.000000,0.529412,0.586184,0.366222,...,0.556522,0.1250,0.652028,0.483221,0.6,0.0,4.48739,0,0,NaN
1,3,2,0.008,0.0,0.0,0.241379,0.512077,0.235294,0.686184,0.235556,...,0.704348,0.3750,0.143455,0.751678,0.2,0.0,4.48739,0,0,NaN
2,3,3,0.016,0.0,0.0,0.172414,0.864734,0.294118,0.594079,0.088000,...,0.482609,0.0625,0.641154,0.530201,0.8,0.0,4.48739,0,0,NaN
3,3,4,0.024,0.0,0.0,0.620690,0.705314,0.294118,0.799342,0.426667,...,0.517391,0.1875,0.542451,0.416107,0.4,0.0,4.48739,0,0,NaN
4,3,5,0.032,0.0,0.0,0.413793,0.599034,0.294118,0.841447,0.552889,...,0.647826,0.6875,0.695943,0.463087,0.2,0.0,4.48739,0,0,NaN
5,3,6,0.040,0.0,0.0,0.000000,0.869565,0.352941,0.659211,0.120889,...,0.700000,0.3125,0.646173,0.516779,0.6,0.0,4.48739,0,0,NaN
6,3,7,0.048,0.0,0.0,0.568966,0.801932,0.294118,0.337500,0.463111,...,0.521739,0.0625,0.808030,0.429530,0.2,0.0,4.48739,0,0,NaN
7,3,8,0.056,0.0,0.0,0.344828,0.811594,0.235294,0.623026,0.629333,...,0.626087,0.2500,0.190715,0.711409,0.5,0.0,4.48739,0,0,NaN
8,3,9,0.064,0.0,0.0,0.379310,0.555556,0.235294,0.953947,0.421333,...,0.417391,0.6250,0.293183,0.248322,0.3,0.0,4.48739,0,0,NaN
9,3,10,0.072,0.0,0.0,0.431034,0.772947,0.411765,0.738158,0.432889,...,0.717391,0.2500,0.712254,0.604027,0.7,0.0,4.48739,0,0,NaN
